<a href="https://colab.research.google.com/github/MEskerod/project_deeplearning_2023/blob/main/deeplearning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

#Import files

In [ ]:
import zipfile

# Define the path to the zipped folder in your Google Drive
zip_file_path = '/content/drive/My Drive/path_to_your_folder.zip' #Change!

# Define the folder where you want to unzip the files (in-memory)
unzip_folder = '/content/data' #Change name?!

# Create the destination folder (in-memory)
import os
os.makedirs(unzip_folder, exist_ok=True)

# Unzip the folder
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_folder)

# List the files in the unzipped folder
os.listdir(unzip_folder)

#Set up preprocessing of RNA files